<a href="https://colab.research.google.com/github/Alvise84/machine_learning/blob/main/RNN_Vanilla_poezia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Генерация поэзии с помощью нейронных сетей:**

Задача: генерировать стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.



## Шаг 1: Установка необходимых библиотек

Установим необходимые библиотеки, если они еще не установлены.

In [ ]:
# Установка необходимых библиотек
!pip install torch

## Шаг 2: Импорт библиотек

Импортируем все необходимые библиотеки.

In [ ]:
# Импорт библиотек
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Шаг 3: Определение устройства

Определим, будет ли использоваться GPU или CPU для обучения модели.

In [ ]:
# Определение устройства
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))

## Шаг 4: Загрузка данных

Загрузим текст "Евгений Онегин" и подготовим его для дальнейшей обработки.

In [ ]:
# Загрузка данных
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt

with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()

text = "".join([x.replace('\t\t', '').lower() for x in text])

## Шаг 5: Построение словаря

Построим словарь символов и создадим маппинг между символами и их индексами.

In [ ]:
# Построение словаря
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")

text_encoded = [token_to_idx[x] for x in text]

## Шаг 6: Генерация случайного батча

Определим функцию для генерации случайных батчей данных.

In [ ]:
# Генерация случайного батча
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))

## Шаг 7: Определение модели Vanilla RNN

Определим архитектуру модели Vanilla RNN.

In [ ]:
# Определение модели Vanilla RNN
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)

## Шаг 8: Определение гиперпараметров

Определим гиперпараметры для обучения модели.

In [ ]:
# Гиперпараметры
input_size = num_tokens
hidden_size = 128
output_size = num_tokens
learning_rate = 0.01
num_epochs = 100
batch_size = 256
seq_length = 100

## Шаг 9: Создание экземпляра модели, функции потерь и оптимизатора

Создадим экземпляр модели, функцию потерь и оптимизатор.

In [ ]:
# Создание экземпляра модели, функции потерь и оптимизатора
model = VanillaRNN(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Шаг 10: Обучение модели

Обучим модель на данных.

In [ ]:
# Обучение модели
for epoch in range(num_epochs):
    for chunk in generate_chunk():
        inputs = torch.tensor(chunk[:, :-1], dtype=torch.long).to(device)
        targets = torch.tensor(chunk[:, 1:], dtype=torch.long).to(device)

        # One-hot encoding
        inputs_one_hot = F.one_hot(inputs, num_classes=num_tokens).float()

        hidden = model.init_hidden(batch_size)
        outputs, _ = model(inputs_one_hot, hidden)

        loss = criterion(outputs.view(-1, num_tokens), targets.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

## Шаг 11: Функция генерации текста

Определим функцию для генерации текста на основе обученной модели.

In [ ]:
# Функция генерации текста
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else:
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)

    # Инициализация скрытого состояния
    hidden = char_rnn.init_hidden(1)

    # Ввод начальной фразы в модель
    with torch.no_grad():
        for i in range(len(x_sequence[0]) - 1):
            input_tensor = x_sequence[:, i].unsqueeze(1)
            input_one_hot = F.one_hot(input_tensor, num_classes=num_tokens).float()
            _, hidden = char_rnn(input_one_hot, hidden)

        # Генерация новых символов
        for _ in range(max_length - len(x_sequence[0])):
            input_tensor = x_sequence[:, -1].unsqueeze(1)
            input_one_hot = F.one_hot(input_tensor, num_classes=num_tokens).float()
            output, hidden = char_rnn(input_one_hot, hidden)

            # Применение температуры
            output_probs = F.softmax(output[0] / temperature, dim=-1)
            next_token = torch.multinomial(output_probs, 1).item()

            # Добавление следующего символа к последовательности
            x_sequence = torch.cat((x_sequence, torch.tensor([[next_token]], dtype=torch.int64).to(device)), dim=1)

    # Преобразование индексов обратно в символы
    generated_text = ''.join([idx_to_token[idx] for idx in x_sequence.cpu().data.numpy()[0]])

    return generated_text

## Шаг 12: Генерация десяти последовательностей

Сгенерируем десять последовательностей длиной 500 символов, начиная с фразы " мой дядя самых честных правил".

In [ ]:
# Генерация десяти последовательностей
seed_phrase = ' мой дядя самых честных правил'
generated_phrases = [
    generate_sample(
        model,
        seed_phrase,
        max_length=500,
        temperature=0.8
    ).replace('<sos>', '')  # Удаляем токен <sos>
    for _ in range(10)
]

## Шаг 13: Проверка и сохранение результатов

Проверим, что все сгенерированные фразы имеют длину 500 символов, и сохраним результаты в файл submission_dict.json.

In [ ]:
# Проверка и сохранение результатов
import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:
    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")
    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")
    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')